<a href="https://colab.research.google.com/github/Omarnot2far/End-to-End-DataScience-project-internship/blob/main/Clean_data_BritishAirline_Reveiw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
pip install beautifulsoup4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Web Scrapping

In [128]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Define the URL pattern
url_pattern = 'https://www.airlinequality.com/airline-reviews/british-airways/page/{}/'

# Initialize empty lists to store the data
data = []

# Loop through all the pages
for page in range(1, 101):
    # Make a request to the webpage
    url = url_pattern.format(page)
    response = requests.get(url)
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the reviews on the page
    reviews = soup.find_all('article', {'itemtype': 'http://schema.org/Review'})
    
    # Loop through the reviews and extract the data
    for review in reviews:
        rating = review.find('span', {'itemprop': 'ratingValue'}).text
        headline = review.find('h2', class_='text_header').text
        author = review.find('span', {'itemprop': 'author'}).text
        time = review.find('time', {'itemprop': 'datePublished'}).text
        location = review.find('h3', class_='text_sub_header userStatusWrapper').text
        content = review.find('div', class_='text_content').text

        data.append({'rating': rating, 'headline': headline, 'author' : author, 'time': time, 'location': location, 'content': content})

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data, columns=['rating', 'headline', 'author', 'time', 'location', 'content'])
print(df)


    rating                                  headline              author  \
0        1          "most appalling airline service"           \nC Peale   
1        1            "Customer service is shocking"     \nJason Wickert   
2        1                     "Avoid this airline"          \nS Dasirou   
3        2         "a lot of actual lies being told"  \nAlexander George   
4        5  "premium price for very average service"           \nG Jones   
..     ...                                       ...                 ...   
995      1               "first and last experience"           \nH Reyes   
996      1       "charged £65 to bring my bike back"        \nMark Howes   
997      1          "queue for bag drop was massive"            \nL Wang   
998      1                       "they did not care"        \nDavid Moss   
999      8           "always had a great experience"   \nKirsten Durward   

                time                                           location  \
0       8th 

In [129]:
df.head()

,rating,headline,author,time,location,content
0,1,"""most appalling airline service""",\nC Peale,8th May 2023,\n\nC Peale (Australia) 8th May 2023,✅ Trip Verified | I will never travel with Br...
1,1,"""Customer service is shocking""",\nJason Wickert,6th May 2023,\n\nJason Wickert (United Kingdom) 6th May 2023,✅ Trip Verified | I am already in Portugal so...
2,1,"""Avoid this airline""",\nS Dasirou,6th May 2023,\n\nS Dasirou (United Kingdom) 6th May 2023,✅ Trip Verified | Terrible. Avoid this airlin...
3,2,"""a lot of actual lies being told""",\nAlexander George,1st May 2023,\n\nAlexander George (United Kingdom) 1st May ...,✅ Trip Verified | Despite being a gold member...
4,5,"""premium price for very average service""",\nG Jones,28th April 2023,\n\nG Jones (United Kingdom) 28th April 2023,Not Verified | Regarding the aircraft and seat...


In [130]:
df.shape

(1000, 6)

# Data Cleaning

In [131]:
df = df.replace({'author': '\n', 'location': '\n', 'content': '\n'}, '', regex=True)


In [132]:
import re

df['location_info'] = df['location'].str.extract(r'\((.*?)\)', expand=False)


In [133]:
df = df.drop('location', axis=1)

In [134]:
df[['trip_varified', 'content']] = df['content'].str.split('|', expand=True)


In [135]:
df.head()

,rating,headline,author,time,content,location_info,trip_varified
0,1,"""most appalling airline service""",C Peale,8th May 2023,I will never travel with British Airways aga...,Australia,✅ Trip Verified
1,1,"""Customer service is shocking""",Jason Wickert,6th May 2023,I am already in Portugal so contacted them t...,United Kingdom,✅ Trip Verified
2,1,"""Avoid this airline""",S Dasirou,6th May 2023,Terrible. Avoid this airline. It’s now becom...,United Kingdom,✅ Trip Verified
3,2,"""a lot of actual lies being told""",Alexander George,1st May 2023,"Despite being a gold member, the British Air...",United Kingdom,✅ Trip Verified
4,5,"""premium price for very average service""",G Jones,28th April 2023,Regarding the aircraft and seat: The business...,United Kingdom,Not Verified


In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   rating         1000 non-null   object
 1   headline       1000 non-null   object
 2   author         1000 non-null   object
 3   time           1000 non-null   object
 4   content        1000 non-null   object
 5   location_info  1000 non-null   object
 6   trip_varified  1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [137]:
df['rating'] = df['rating'].astype(float)
df['time'] = pd.to_datetime(df['time'])

In [138]:
df['trip_verified'] = df['trip_varified'].str.contains('✅ Trip Verified')
df['trip_verified'] = df['trip_verified'].astype(bool)
df = df.drop('trip_varified', axis=1)


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rating         1000 non-null   float64       
 1   headline       1000 non-null   object        
 2   author         1000 non-null   object        
 3   time           1000 non-null   datetime64[ns]
 4   content        1000 non-null   object        
 5   location_info  1000 non-null   object        
 6   trip_verified  1000 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(1), object(4)
memory usage: 48.0+ KB


In [141]:
df['headline'] = df['headline'].str.replace('"', '')


In [142]:
df.head()

,rating,headline,author,time,content,location_info,trip_verified
0,1.0,most appalling airline service,C Peale,2023-05-08,I will never travel with British Airways aga...,Australia,True
1,1.0,Customer service is shocking,Jason Wickert,2023-05-06,I am already in Portugal so contacted them t...,United Kingdom,True
2,1.0,Avoid this airline,S Dasirou,2023-05-06,Terrible. Avoid this airline. It’s now becom...,United Kingdom,True
3,2.0,a lot of actual lies being told,Alexander George,2023-05-01,"Despite being a gold member, the British Air...",United Kingdom,True
4,5.0,premium price for very average service,G Jones,2023-04-28,Regarding the aircraft and seat: The business...,United Kingdom,False


In [143]:
# save the DataFrame as a CSV file
df.to_csv('clea_BA_data.csv', index=False)